# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [63]:
# import libraries
import re
import numpy as np
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
import pickle
from sqlalchemy import create_engine
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC

In [13]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql_table('messages', engine)

X = df['message']
Y = df[df.columns[4:]]

In [14]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [15]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
Y.apply(pd.value_counts)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,6110,21666,25996,15308,24038,24804,25390,25642,25256,26113.0,...,25804,24966,18848,23974,23681,25831,23664,25586,24740,21064
1,20003,4447,117,10805,2075,1309,723,471,857,NaN,...,309,1147,7265,2139,2432,282,2449,527,1373,5049


### 2. Write a tokenization function to process your text data

In [17]:
def tokenize(text):
    """ Tokenize a text into a list of stemmatized words. """
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    
    return [lemmatizer.lemmatize(token).strip() for token in tokens if token not in stopwords.words("english")]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [20]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier(), n_jobs=1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x00000202F62E9378>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorith

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
y_preds = pipeline.predict(X_test)

In [23]:
def show_results(y_test, y_preds):
    average_f1, average_recall, average_precision = [], [], []
    for idx, column in enumerate(y_test.columns.tolist()):
        print('Column: ' + column + '\n')
        print(classification_report(y_test.values[:, idx], y_preds[:, idx]))
        average_f1.append(f1_score(y_test.values[:, idx], y_preds[:, idx], average='weighted'))
        average_recall.append(recall_score(y_test.values[:, idx], y_preds[:, idx], average='weighted'))
        average_precision.append(precision_score(y_test.values[:, idx], y_preds[:, idx], average='weighted'))
        
    return average_f1, average_recall, average_precision

In [24]:
average_f1, average_recall, average_precision = show_results(y_test, y_preds)

Column: related

              precision    recall  f1-score   support

           0       0.65      0.20      0.31      1532
           1       0.80      0.97      0.87      4997

    accuracy                           0.79      6529
   macro avg       0.73      0.59      0.59      6529
weighted avg       0.76      0.79      0.74      6529

Column: request

              precision    recall  f1-score   support

           0       0.90      0.97      0.93      5415
           1       0.76      0.50      0.60      1114

    accuracy                           0.89      6529
   macro avg       0.83      0.73      0.77      6529
weighted avg       0.88      0.89      0.88      6529

Column: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6500
           1       0.00      0.00      0.00        29

    accuracy                           0.99      6529
   macro avg       0.50      0.50      0.50      6529
weighted avg       0.99  

Column: fire

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6455
           1       0.52      0.36      0.43        74

    accuracy                           0.99      6529
   macro avg       0.76      0.68      0.71      6529
weighted avg       0.99      0.99      0.99      6529

Column: earthquake

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5900
           1       0.86      0.79      0.82       629

    accuracy                           0.97      6529
   macro avg       0.92      0.89      0.90      6529
weighted avg       0.97      0.97      0.97      6529

Column: cold

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6401
           1       0.71      0.36      0.48       128

    accuracy                           0.98      6529
   macro avg       0.85      0.68      0.73      6529
weighted avg       0.98   

In [25]:
print('Average f1 score: ', np.mean(average_f1))
print('Average precision: ', np.mean(average_precision))
print('Average recall: ', np.mean(average_recall))

Average f1 score:  0.9408314683241018
Average precision:  0.9407826078064611
Average recall:  0.9481756607273533


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
pipeline.get_params().keys()

dict_keys(['tfidf', 'verbose', 'vect__strip_accents', 'vect__token_pattern', 'tfidf__sublinear_tf', 'vect__max_df', 'vect__encoding', 'classifier__estimator', 'tfidf__smooth_idf', 'classifier__estimator__n_estimators', 'steps', 'classifier__estimator__base_estimator', 'memory', 'classifier__n_jobs', 'vect__stop_words', 'classifier__estimator__algorithm', 'tfidf__use_idf', 'vect__max_features', 'vect__decode_error', 'classifier__estimator__learning_rate', 'vect__tokenizer', 'vect__preprocessor', 'vect__input', 'tfidf__norm', 'classifier', 'vect__lowercase', 'vect__binary', 'vect__ngram_range', 'classifier__estimator__random_state', 'vect__min_df', 'vect__vocabulary', 'vect__analyzer', 'vect__dtype', 'vect'])

In [27]:
random_state = 42
parameters = {'classifier__estimator__n_estimators':[20, 30, 50, 100], 
              'classifier__estimator__learning_rate':[0.1, 0.2, 0.5, 1.0], 
              'classifier__estimator__algorithm': ('SAMME', 'SAMME.R'),
              'classifier__estimator__random_state': [random_state]}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
cv.fit(X_train, y_train)

c:\users\benjamin\.virtualenvs\disaster_response_project-jmxca6tq\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [29]:
y_preds = cv.predict(X_test)

average_f1, average_recall, average_precision = show_results(y_test, y_preds)

Column: related

              precision    recall  f1-score   support

           0       0.73      0.23      0.35      1532
           1       0.81      0.97      0.88      4997

    accuracy                           0.80      6529
   macro avg       0.77      0.60      0.62      6529
weighted avg       0.79      0.80      0.76      6529

Column: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5415
           1       0.81      0.47      0.60      1114

    accuracy                           0.89      6529
   macro avg       0.86      0.73      0.77      6529
weighted avg       0.89      0.89      0.88      6529

Column: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6500
           1       0.00      0.00      0.00        29

    accuracy                           0.99      6529
   macro avg       0.50      0.50      0.50      6529
weighted avg       0.99  

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6465
           1       0.25      0.03      0.06        64

    accuracy                           0.99      6529
   macro avg       0.62      0.52      0.53      6529
weighted avg       0.98      0.99      0.99      6529

Column: other_infrastructure

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6260
           1       0.48      0.05      0.09       269

    accuracy                           0.96      6529
   macro avg       0.72      0.52      0.53      6529
weighted avg       0.94      0.96      0.94      6529

Column: weather_related

              precision    recall  f1-score   support

           0       0.87      0.96      0.92      4689
           1       0.88      0.64      0.74      1840

    accuracy                           0.87      6529
   macro avg       0.87      0.80      0.83      6529
weighted avg       

In [30]:
print('Average f1 score: ', np.mean(average_f1))
print('Average precision: ', np.mean(average_precision))
print('Average recall: ', np.mean(average_recall))

Average f1 score:  0.9403448760647556
Average precision:  0.9428144631545098
Average recall:  0.9494562720171541


In [55]:
best_model = cv.best_estimator_
best_model

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x00000202F62E9378>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorith

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 8.1 Random Forest

In [56]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=100), n_jobs=1))
])

In [57]:
pipeline.fit(X_train, y_train)
y_preds = pipeline.predict(X_test)
average_f1, average_recall, average_precision = show_results(y_test, y_preds)

Column: related

              precision    recall  f1-score   support

           0       0.70      0.40      0.51      1532
           1       0.84      0.95      0.89      4997

    accuracy                           0.82      6529
   macro avg       0.77      0.67      0.70      6529
weighted avg       0.81      0.82      0.80      6529

Column: request

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5415
           1       0.85      0.50      0.63      1114

    accuracy                           0.90      6529
   macro avg       0.88      0.74      0.79      6529
weighted avg       0.90      0.90      0.89      6529

Column: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6500
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6529
   macro avg       0.50      0.50      0.50      6529
weighted avg       0.99  

c:\users\benjamin\.virtualenvs\disaster_response_project-jmxca6tq\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\benjamin\.virtualenvs\disaster_response_project-jmxca6tq\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\benjamin\.virtualenvs\disaster_response_project-jmxca6tq\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5803
           1       0.85      0.59      0.69       726

    accuracy                           0.94      6529
   macro avg       0.90      0.79      0.83      6529
weighted avg       0.94      0.94      0.94      6529

Column: shelter

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5973
           1       0.82      0.38      0.52       556

    accuracy                           0.94      6529
   macro avg       0.88      0.69      0.74      6529
weighted avg       0.93      0.94      0.93      6529

Column: clothing

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6426
           1       0.67      0.06      0.11       103

    accuracy                           0.98      6529
   macro avg       0.83      0.53      0.55      6529
weighted avg       0.98      0.98      

Column: direct_report

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      5243
           1       0.80      0.35      0.49      1286

    accuracy                           0.85      6529
   macro avg       0.83      0.66      0.70      6529
weighted avg       0.85      0.85      0.83      6529



In [58]:
print('Average f1 score: ', np.mean(average_f1))
print('Average precision: ', np.mean(average_precision))
print('Average recall: ', np.mean(average_recall))

Average f1 score:  0.937648655253112
Average precision:  0.9393347463696811
Average recall:  0.9493413999081023


As we can see, the performance of the RandomRorestClassifier is a bit lower than the performance of the AdaboostClassifier with default values. Therfore, we will keep the Adaboost as our best model for now. 

### 9. Export your model as a pickle file

In [68]:
with open('model.pkl', 'wb') as pickle_file:
    pickle.dump(best_model, pickle_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.